## Avaliação do Modelo em Produção
### A partir das previsões realizadas pelo modelo mais efetivo com os dados atualizados até o dia 18/04

# 

## Importando Módulos

In [116]:
import pandas as pd
import numpy as np

## Load Data

In [117]:
#Carregando dados de forecast
forecast = pd.read_csv("forecast_18042020.csv",sep=";")
forecast.date = pd.to_datetime(forecast.date)
forecast

,date,state,UF,cases
0,2020-01-30,Acre,AC,0.000000
1,2020-01-31,Acre,AC,0.000000
2,2020-02-01,Acre,AC,0.000000
3,2020-02-02,Acre,AC,0.000000
4,2020-02-03,Acre,AC,0.000000
...,...,...,...,...
2344,2020-04-21,São Paulo,SP,17363.892890
2345,2020-04-22,São Paulo,SP,18702.456607
2346,2020-04-23,São Paulo,SP,20143.764813
2347,2020-04-24,São Paulo,SP,21695.703893


In [118]:
#Carregando dados atuais
att = pd.read_csv("attcovid19.csv",sep=",")
att.date = pd.to_datetime(att.date)
att

,date,region,state,cases,deaths
0,2020-01-30,Centro-Oeste,Distrito Federal,0,0
1,2020-01-30,Centro-Oeste,Goiás,0,0
2,2020-01-30,Centro-Oeste,Mato Grosso,0,0
3,2020-01-30,Centro-Oeste,Mato Grosso do Sul,0,0
4,2020-01-30,Nordeste,Alagoas,0,0
...,...,...,...,...,...
2236,2020-04-21,Sudeste,Rio de Janeiro,5306,461
2237,2020-04-21,Sudeste,São Paulo,15385,1093
2238,2020-04-21,Sul,Paraná,1024,51
2239,2020-04-21,Sul,Rio Grande do Sul,904,27


## Processing Data

In [119]:
#Selecionando dados apenas após o primeiro caso confirmado de COVID19 no Brasil
subset = att.groupby('date')[['cases']].sum()
start = subset[subset['cases']!=0].head(1).index[0]

forecast = forecast[forecast['date'] >= start].reset_index(drop=True)
att = att[att['date']>start].reset_index(drop=True)

#Selecionando a intersecção dos datasets
date_max = att.date.max()
forecast = forecast[forecast['date'] <= date_max].reset_index(drop=True)
forecast.tail()

,date,state,UF,cases
1507,2020-04-20,Rio de Janeiro,RJ,5279.142031
1508,2020-04-21,Rio de Janeiro,RJ,5690.113335
1509,2020-04-19,São Paulo,SP,14966.224232
1510,2020-04-20,São Paulo,SP,16120.749462
1511,2020-04-21,São Paulo,SP,17363.892890


In [120]:
forecast = forecast.groupby(['state','date'])[['cases']].sum().reset_index()
forecast.tail(4)

,state,date,cases
1508,Tocantins,2020-04-18,33.000000
1509,Tocantins,2020-04-19,35.585867
1510,Tocantins,2020-04-20,38.370217
1511,Tocantins,2020-04-21,41.368287


In [121]:
att = att.groupby(['state','date'])[['cases']].sum().reset_index()
att.tail(4)

,state,date,cases
1481,Tocantins,2020-04-18,33
1482,Tocantins,2020-04-19,33
1483,Tocantins,2020-04-20,34
1484,Tocantins,2020-04-21,37


## Avaliação do Modelo em Produção
### Métricas - RMSLE & MAE

In [122]:
#Root Mean Squared Logarithmic Error
def RMSLE(pred,target):
    
    pred=np.array(pred)
    target=np.array(target)
    
    return np.sqrt(np.mean((np.log1p(abs(pred))-np.log1p(abs(target)))**2))

#Mean Absolute Error
def MAE(pred,target):
    
    pred=np.array(pred)
    target=np.array(target)
    
    return np.mean(abs(pred-target))

In [123]:
#Dia de início das previsões
foredate = '2020-04-19'

In [124]:
#Avaliação geral por estado e dia

#Separando dados preditos e verdadeiros
ytrue = att[att['date'] >= foredate].cases
ypred = forecast[forecast['date'] >= foredate].cases

print("A previsão do modelo para cada estado por dia possui um RMSLE de {:.2f}.\nSendo que há um erro médio de +- {:.0f} casos confirmados por dia em cada estado.".
     format(RMSLE(ypred,ytrue),MAE(ypred,ytrue)))

A previsão do modelo para cada estado por dia possui um RMSLE de 0.09.
Sendo que há um erro médio de +- 107 casos confirmados por dia em cada estado.


In [125]:
#Avaliação da soma dos casos no Brasil

#Separando dados preditos e verdadeiros
ytrue = att[att['date'] >= foredate].groupby('date')['cases'].sum()
ypred = forecast[forecast['date'] >= foredate].groupby('date')['cases'].sum()


print("A previsão do modelo para o Brasil possui um RMSLE de {:.2f}.\nSendo que há um erro médio de +- {:.0f} casos confirmados por dia no país.".
     format(RMSLE(ypred,ytrue),MAE(ypred,ytrue)))

A previsão do modelo para o Brasil possui um RMSLE de 0.05.
Sendo que há um erro médio de +- 1999 casos confirmados por dia no país.


In [126]:
#Avaliação específica por Estado

states = att.state.unique().tolist()
avaliation = pd.DataFrame(columns=['Estado', 'MAE', 'RMSLE'])

print("Estado - MAE - RMSLE\n")
for i in range(0,len(states)):
    
    ytrue = att[att['state'] == states[i]]
    ypred = forecast[forecast['state'] == states[i]]
    
    ytrue = ytrue[ytrue['date'] >= foredate].cases
    ypred = ypred[ypred['date'] >= foredate].cases
    
    print("{} - {:.2f} - {:.2f}".format(states[i],MAE(ypred,ytrue),RMSLE(ypred,ytrue)))
    
    new = pd.DataFrame({'Estado':states[i],'MAE':MAE(ypred,ytrue),'RMSLE':RMSLE(ypred,ytrue)}, index=[0])
    
    avaliation = pd.concat([avaliation,new]).reset_index(drop=True)

Estado - MAE - RMSLE

Acre - 12.09 - 0.07
Alagoas - 25.34 - 0.16
Amapá - 33.26 - 0.08
Amazonas - 57.52 - 0.03
Bahia - 44.48 - 0.04
Ceará - 52.84 - 0.02
Distrito Federal - 37.70 - 0.06
Espírito Santo - 41.78 - 0.05
Goiás - 35.64 - 0.09
Maranhão - 86.90 - 0.07
Mato Grosso - 32.07 - 0.17
Mato Grosso do Sul - 16.71 - 0.10
Minas Gerais - 72.08 - 0.07
Paraná - 103.76 - 0.11
Paraíba - 8.57 - 0.04
Pará - 122.10 - 0.17
Pernambuco - 126.33 - 0.05
Piauí - 8.01 - 0.05
Rio Grande do Norte - 33.46 - 0.06
Rio Grande do Sul - 95.01 - 0.11
Rio de Janeiro - 298.91 - 0.06
Rondônia - 22.45 - 0.16
Roraima - 7.92 - 0.05
Santa Catarina - 108.50 - 0.10
Sergipe - 4.57 - 0.06
São Paulo - 1406.29 - 0.09
Tocantins - 3.77 - 0.10


In [127]:
#Ordenando os estados com menor RMSLE
avaliation.sort_values(by='RMSLE').style.background_gradient(cmap="summer_r",vmin=0,vmax=0.20)

,Estado,MAE,RMSLE
5,Ceará,52.839191,0.016331
3,Amazonas,57.523327,0.032008
14,Paraíba,8.567368,0.035874
4,Bahia,44.479322,0.036575
7,Espírito Santo,41.780534,0.045471
16,Pernambuco,126.333170,0.048247
22,Roraima,7.917314,0.050334
17,Piauí,8.014645,0.053658
6,Distrito Federal,37.698613,0.057525
24,Sergipe,4.565305,0.057528
